# Model Selection 모듈 소개

## 학습/데이터 분리 - train_test_split()

### 사이킷런 model_selection 모듈의 주요 기능

- 학습데이터와 테스트 데이터 분리

- 교차 검증 분할 및 평가

- Estimator의 하이퍼 파라미터 튜닝

#### 학습 데이터와 테스트 데이터 분리 : 
#### train_test_split(feature_dataset, label_dataset, test_size, train_size, radom_state, shuffle, stratify)

- test_size : 전체DB에서 Test DB 크기를 몇으로 샘플링 할 것인가
    - 0-1사이, train_size 둘중 하나만 쓰면 됨
    - 디폴트는 0.25 = 25%
    - train_size는 잘 안씀 test_size를 많이 씀
    
    
- shuffle : 데이터 분할 전 섞을지 결정. 디폴트=True(보통은 default값으로)


- random_state : 호출할 때마다 동일 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값
    - train_test_split()은 무작위로 데이터 분리, random_state를 지정하지 않으면 다른 학습/테스트용 데이터 생성


- stratify : 지정된 label의 클래스 비율에 맞게 분할 ``classification 다룰 때 중요``
    - default = None
    - stratify 값을 target으로 지정해주면 각가의 class 비율(ratio)을 train/validation에 유지


- train_test_split() 반환값 : 
    - X_train : 학습용 데이터 feature
    - X_test : 테스트용 데이터 feature
    - y_train : 학습용 데이터 target
    - y_test : 테스트용 데이터 target
    - feature : 대문자 X, target : 소문자 y

In [1]:
# 테스트 데이터 셋 이용 하지 않고 예측
# 학습 데이터 세트만 이용했기 때문에 정확도 100%
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

train_data = iris_data.data
train_label = iris_data.target

#학습 : fit()
dt_clf.fit(train_data,train_label)

#예측 predict()
pred = dt_clf.predict(train_data)
print("예측 정확도: ", accuracy_score(train_label,pred) )

예측 정확도:  1.0


In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

#학습/테스트 분리

X_train,X_test,y_train,y_test=train_test_split(iris_data.data, iris_data.target, test_size=0.3, random_state=21)

print(y_train)

[0 0 0 1 0 1 0 1 2 1 1 0 0 2 0 0 0 0 0 1 1 0 2 1 0 0 2 1 0 2 1 2 0 2 0 0 1
 2 1 1 0 2 1 0 2 1 1 2 1 2 1 2 2 0 0 2 2 0 1 2 1 1 2 1 2 0 2 2 0 0 1 2 0 0
 1 2 0 0 1 2 2 0 2 1 0 1 2 1 0 2 2 1 1 2 2 2 1 2 1 1 2 2 0 0 1]


In [3]:
#학습수행
dt_clf.fit(X_train,y_train)
#예측수행
pred = dt_clf.predict(X_test)
print("예측정확도 : ", accuracy_score(y_test,pred))

예측정확도 :  0.9333333333333333


In [4]:
import pandas as pd
iris_df=pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target']=iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
# 피처 데이터 프레임 반환 : 마지막 열 전까지
feature_df = iris_df.iloc[:,:-1]
# 타겟 데이터 프레임 반환
target_df = iris_df.iloc[:,-1]
#학습/테스트 데이터 분할
X_train,X_test,y_train,y_test=train_test_split(feature_df, target_df, test_size=0.3, random_state=13)

In [6]:
type(X_train)

pandas.core.frame.DataFrame

In [7]:

dt_clt.fit(X_train, y_train)
# 테스트데이터로 예측
pred = dt_clt.predict(X_test)
print('예측정확도: ', accuracy_score(y_test,pred))

NameError: name 'dt_clt' is not defined

# data split과 모델 검증

- 홀드아웃, 교차검증을 많이 이용, 필요에 따라Stratified Sampling

## 홀드아웃 방식
- 트레인과 테스트 세트로 나누어서 7:3~ 9:1로 사용함

- 학습/테스트 데이터 분리 시 문제
    - 과적합(Overfitting) 문제 발생
    - 부적합한 데이터 선별로 알고리즘 성능 저하
    
- 과적합 문제 발생
    - 고정된 학습 데이터와 테스트 데이터에만 최적의 성능을 발휘할 수 있도록

- 문제 개선 => 교차검증(Cross Validation,CV)

## 교차검증(Cross Validation,CV)

- 교차검증 방법

    - K 폴드 교차검증
    - Stratified K 폴드 교차 검증
    
- K폴드 교차검증

    - K개의 데이터 폴드 세트를 만듬
    - K번 만큼 각 폴드 세트에 학습과 검증 평가를 반복 수행
    - 가장 보편적인 교차검증 기법
    
### K폴드 사이킷런 클래스

- KFold 클래스 : kfold = KFold(n_splits=5)

- split()메소드 : 

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

iris = load_iris()
features = iris.data
label = iris.target

features.shape

(150, 4)

In [22]:
# DecisionTreeClassifier 객체 생성
dct = DecisionTreeClassifier(random_state=156)

#5개의 폴드 세트로 분리하는 kfold 객체 생성
kfold = KFold(n_splits=5)

# 폴드 세트별 정확도를 담을 리스트 객체 생성
cv_accuracy = []


In [23]:
#폴드 별 학습용 , 검증용 데이터 세트의 행 인덱스 확인
for train_index, test_index in kfold.split(features):
    print(train_index, test_index)

[ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 1

In [24]:
import numpy as np
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train = label[train_index]
    y_test = label[test_index]
    
    dct.fit(X_train, y_train)
    pred = dct.predict(X_test)
   
    # 라운드 함수를 써서 정확도 확인 시 소수점 3자리까지 출력
    acc = np.round(accuracy_score(y_test,pred),3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('정확도: %f ,학습데이터 크기 : %d, 검증데이터 크기 : %d' %(acc, train_size, test_size))
    cv_accuracy.append(acc)
    
print('평균 검증 정확도:', np.mean(cv_accuracy))

정확도: 1.000000 ,학습데이터 크기 : 120, 검증데이터 크기 : 30
정확도: 0.967000 ,학습데이터 크기 : 120, 검증데이터 크기 : 30
정확도: 0.867000 ,학습데이터 크기 : 120, 검증데이터 크기 : 30
정확도: 0.933000 ,학습데이터 크기 : 120, 검증데이터 크기 : 30
정확도: 0.733000 ,학습데이터 크기 : 120, 검증데이터 크기 : 30
평균 검증 정확도: 0.9


### Stratified K 폴드 교차검증

- 불균형한 분포도를 가진 레이블 데이터집합을 위한 K 폴드 방식

#### 불균형 데이터(imbalanced data)

- 관심 대상 데이터가 상대적으로 매우 적은 비율로 나타나는 데이터 문제

- 분류 문제인 경우 : 클래스들이 균일하게 분포하지 않은 문제를 의미
    - ex) 불량률이 1%인 생산라인에서 양품과 불량품을 예측
    - 사기감지탐지, 이상거래감지, 의료진단 등에서 자주 나타남

- 회귀 문제인 경우 : 극단값이 포함되어 있는 치우친 데이터 사례
    - ex)산불에 의한 피해 면적 예측

극복하는 방법
- 데이터 추가 확보
- Re-Sampling
    - 과소표집(Under-Sampling)
        - 다른 클래스에 비해 상대적으로 많이 나타나는 클래스 개수를 줄임
        - 균형은 유지할 수 있느나 유용한 정보의 손실
    - 과대표집(Over-Sampling)
        - 상대적으로 적게 나타나는 클래스의 데이터를 복제해 데이터 갯수를 늘림
        - 정보 손실은 없으나 과적합 위험
        - 이를 회피하기 위해 SMOTE 같이 임의의 값을 생성 후 추가
        

In [25]:
import pandas as pd
iris_df=pd.DataFrame(data = iris_data, columns=iris_data.feature_names)
iris_df['label']=iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,0


In [26]:
# 빈수도 확인
iris_df['label'].value_counts()
# 0 : satosa, 1: versicolor, 2: virginica

0    50
1    50
2    50
Name: label, dtype: int64

In [27]:
# 3개의 폴드 구성
kfold = KFold(n_splits=3)
n=0
for train_index, test_index in kfold.split(iris_df):
    n+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('[교차검증 : %d]'%(n))
    print('학습용 : ', label_train.value_counts())
    print('검증용 : ', label_test.value_counts())

[교차검증 : 1]
학습용 :  1    50
2    50
Name: label, dtype: int64
검증용 :  0    50
Name: label, dtype: int64
[교차검증 : 2]
학습용 :  0    50
2    50
Name: label, dtype: int64
검증용 :  1    50
Name: label, dtype: int64
[교차검증 : 3]
학습용 :  0    50
1    50
Name: label, dtype: int64
검증용 :  2    50
Name: label, dtype: int64


In [28]:
dct = DecisionTreeClassifier(random_state=156)
kfold = KFold(n_splits=3)
cv_accuracy = []
n=0
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train = label[train_index]
    y_test = label[test_index]
    
    # 학습 및 예측
    dct.fit(X_train, y_train)
    pred = dct.predict(X_test)
    n+=1
    # 라운드 함수를 써서 정확도 확인 시 소수점 3자리까지 출력
    acc = np.round(accuracy_score(y_test,pred),3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('%d \n정확도: %f ,학습데이터 크기 : %d, 검증데이터 크기 : %d' %(n,acc, train_size, test_size))
    cv_accuracy.append(acc)
print('평균 검증 정확도:', np.mean(cv_accuracy))

1 
정확도: 0.000000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
2 
정확도: 0.000000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
3 
정확도: 0.000000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
평균 검증 정확도: 0.0


In [29]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 3)
n=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n=n+1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('[교차검증 : %d]'%(n))
    print('학습용 레이블 분포 : \n', label_train.value_counts())
    print('테스트 레이블 분포 : \n', label_test.value_counts())
    

[교차검증 : 1]
학습용 레이블 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
테스트 레이블 분포 : 
 0    17
1    17
2    16
Name: label, dtype: int64
[교차검증 : 2]
학습용 레이블 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
테스트 레이블 분포 : 
 0    17
2    17
1    16
Name: label, dtype: int64
[교차검증 : 3]
학습용 레이블 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
테스트 레이블 분포 : 
 1    17
2    17
0    16
Name: label, dtype: int64


StratifiedKFold
- 분류문제에서는 k폴드가 아닌 StratifiedKFold를 활용
- 회기 문제는 k폴드

In [30]:
dct = DecisionTreeClassifier(random_state=156)
# 3개의 폴드 세트로 분리하는 stratifiedKFold 객체 생성
skfold = StratifiedKFold(n_splits=3)

cv_accuracy = []
n=0
for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train,y_test = label[train_index],label[test_index]
    
    # 학습 및 예측
    dct.fit(X_train, y_train)
    pred = dct.predict(X_test)
    
    # 반복 할 때마다 정확도 측적
    n+=1
    
    # 라운드 함수를 써서 정확도 확인 시 소수점 3자리까지 출력
    acc = np.round(accuracy_score(y_test,pred),3)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('%d \n정확도: %f ,학습데이터 크기 : %d, 검증데이터 크기 : %d' %(n,acc, train_size, test_size))
    cv_accuracy.append(acc)
    
print('평균 검증 정확도:', np.mean(cv_accuracy))

1 
정확도: 0.980000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
2 
정확도: 0.940000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
3 
정확도: 0.980000 ,학습데이터 크기 : 100, 검증데이터 크기 : 50
평균 검증 정확도: 0.9666666666666667


# 교차검증을 보다 간편하게

## cross_val_score() 함수

- Estimator를 학습(fit), 예측(predict),평가(evaluation) 를 해줌으로 간단하게 교차검증 가능

- cross_val_score(estimator, X,y=None, scoring=None, cv=None)

In [37]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clt = DecisionTreeClassifier(random_state=156)

features = iris.data
label = iris.target
# cv = 폴드를 몇개로 나눌것이냐
scores = cross_val_score(dt_clt, features, label, scoring='accuracy', cv = 3)
print('교차 검증별 정확도 :',scores)
print('평균 검증 정확도 :', np.round(np.mean(scores),4))

교차 검증별 정확도 : [0.98 0.94 0.98]
평균 검증 정확도 : 0.9667


- cross_val_score()는  cv로 지정된 횟수만큼 scoring 파라미터로 지정된 평가 지표로 평가 결과값을 배열로 반환
- 일반적으로 평가 결과값 평균을 평가 수치로 사용

## 교차검증과 최적의 하이퍼 파라미터 튜닝을 한번에

 하이퍼파라미터
    - 머신러닝 알고리즘을 구성하는 요소
    - 이 값들을 조정해 알고리즘의 예측성능 개선
    
### 사이킷 런의 GridSearchCV클래스
- 알고리즘의 하이퍼 파라미터를 순차적으로 변경하면서 최고 성능을 가지는 파라미터를 찾고자 한다면 파라미터의 집합을 만들어 순차적으로 적용하여 최적화

- estimator : classfier, regressor, peipeline
- param_gid : key + 리스트 값을 가지는 딕셔너리(estimator 튜닝을 위한 파라미터)
    - key : 파라미터 명, 리스트값 : 파라미터 값

- scoring : 예측성능 측정할 평가방법

- cv : 교차검증을 위해 분할되는 학습/ 테스트 세트의 개수



In [53]:
# GridSearchCV 

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

#: 데이터셋 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data,iris.target,test_size=0.2,random_state=121)

dt_clf = DecisionTreeClassifier()

# 하이퍼파라미터는 딕셔너리 형식 지정
# key : 결정트리의 하이퍼파라미터
#value : 하이퍼파라미터의 값
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [64]:
grid_tree=GridSearchCV(dt_clt, param_grid=parameters, cv=3, refit=True,return_train_score=True)
#param_grid 의 하이퍼 파라미터를 순차적으로 학습/평가
grid_tree.fit(X_train, y_train)
# gridsearchcv 결과를 추출해 dataframe으로 변환
scores_df = pd.DataFrame(grid_tree.cv_results_)

scores_df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000665,0.00047,0.0,0.0,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.700,0.7000,0.7000,0.700000,1.110223e-16
1,0.000665,0.00047,0.0,0.0,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.700,0.7,0.70,0.700000,1.110223e-16,5,0.700,0.7000,0.7000,0.700000,1.110223e-16
2,0.000665,0.00047,0.0,0.0,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.925,1.0,0.95,0.958333,3.118048e-02,3,0.975,0.9375,0.9625,0.958333,1.559024e-02


In [65]:
scores_df[['params','mean_test_score','rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1


In [66]:
#최고성능을 가지는 파라미터 조합 및 예측성능 1위값 출력
# best_params_
print('최적의 파라미터:',grid_tree.best_params_)
print('최고의 적확도:',grid_tree.best_score_)

최적의 파라미터: {'max_depth': 3, 'min_samples_split': 2}
최고의 적확도: 0.975


In [67]:
best_dt = grid_tree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적학습이 됬으므로 fit으로 별도 학습 안해도됨
pred = best_dt.predict(X_test)

accuracy_score(y_test,pred)

0.9666666666666667

# 데이터 전처리

- 데이터 정제 : 결측치, 이상치, 잡음

- 데이터 결합

- 데이터 변환 : Normalization, scaling

- 차원 축소 
    - Feature selection
        - filter, wrapper, embedded
    - Feature extraction
        - PCA, SVD, FA, NMF

## 데이터 인코딩 : 문자열을 숫자형으로 변환

- 인코딩 방식 : Label encoding, Onehot encoding
    
    - label encoding
        - LabelEncoding 객체 생성
        - fit() : 레이블 인코더를 맞춤
        - transform() : 인코딩된 레이블 반환

In [107]:
from sklearn.preprocessing import LabelEncoder
items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','아이패드','아이패드']


encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코더 반환값 : ',labels)

# 인코딩 전 원래의 값 확인 : encoder.classes_
print('인코딩 전 값 : ',encoder.classes_)

# 인코딩 -> 디코딩 : encoder.inverse_transform
print('3,0,2,1의 디코딩 값 : ',encoder.inverse_transform([3,0,2,1]))

인코더 반환값 :  [0 1 4 5 2 2 3 3]
인코딩 전 값 :  ['TV' '냉장고' '선풍기' '아이패드' '전자레인지' '컴퓨터']
3,0,2,1의 디코딩 값 :  ['아이패드' 'TV' '선풍기' '냉장고']


## 원 - 핫 인코딩 : 회기분석 해야 할 경우 많이 이용함

- 1. 수치형으로 바꾼다 : LabelEncoding
- 2. 2차원 데이터 변환 reshape
- 3. 원-핫 인코딩 적용

In [113]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np

# 1. 숫자값 반환을 위해 LabelEncoder로 변환
items = ['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','아이패드','아이패드']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('label encoding:',labels)

# 2차원 데이터로 변환 : reshape
labels = labels.reshape(-1,1)
print('reshape을 통해 2차원 변환:\n',labels)

# 원-핫 인코딩

onehot = OneHotEncoder()
onehot.fit(labels)
onehot_labels = onehot.transform(labels)
print('원 핫 인코딩 : \n',onehot_labels)


#2차원 데이터로 출력
print('toarray() : 2차원데이터로 출력 : \n',onehot_labels.toarray())

label encoding: [0 1 4 5 2 2 3 3]
reshape을 통해 2차원 변환:
 [[0]
 [1]
 [4]
 [5]
 [2]
 [2]
 [3]
 [3]]
원 핫 인코딩 : 
   (0, 0)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 2)	1.0
  (5, 2)	1.0
  (6, 3)	1.0
  (7, 3)	1.0
toarray() : 2차원데이터로 출력 : 
 [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


## Pandas API 사용 원-핫 인코딩 수행

- get - dummies() 메서드 사용

- 숫자형으로 변환없이 바로 변환

In [87]:
import pandas as pd

df = pd.DataFrame({'item':['TV','냉장고','전자레인지','컴퓨터','선풍기','선풍기','아이패드','아이패드']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_선풍기,item_아이패드,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,1,0,0,0
5,0,0,1,0,0,0
6,0,0,0,1,0,0
7,0,0,0,1,0,0


In [88]:
# Pandas 데이터프레임을 Numpy 배열로 변환
pd.get_dummies(df).to_numpy()

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0]], dtype=uint8)

## 피쳐 스케일링과 정규화
- feature scaling
    - 서로 다른 변수의 값 범위를 일정한 수준으로 맞춤

- 방식
    - 표준화(Standardization) : Z-score
        - 표준화(standardization) : 평균0, 분산1
        - sklearn.preprocessing 의 StandardScaler 모듈
    
    - 정규화(Normalization) : Min-Max
        - 0~1로 변환
        - sklearn.preprocessing의 MinMaxScaler 모듈

In [96]:
# standard scaler
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)
print('feature 들의 평균값 : \n',iris_df.mean())
print('feature 들의 표준편차 : \n',iris_df.std())

feature 들의 평균값 : 
 sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64
feature 들의 표준편차 : 
 sepal length (cm)    0.828066
sepal width (cm)     0.435866
petal length (cm)    1.765298
petal width (cm)     0.762238
dtype: float64


In [101]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)
# print(iris_scaled)

iris_scaled_df = pd.DataFrame(data=iris_scaled, columns = iris.feature_names)
print(iris_scaled_df.mean())
print(iris_scaled_df.std())

sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64
sepal length (cm)    1.00335
sepal width (cm)     1.00335
petal length (cm)    1.00335
petal width (cm)     1.00335
dtype: float64


## MinMaxScaler
- 1. MinMaxScaler 객체 생성
- 2. fit()
- 3. fransform()

In [106]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)
iris_scaled


array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

## 모양변환

- Pivot : 행,열 별 요약된 값으로 정렬해서 분석을 하고자 할 때

- Unpivot : 컬럼 형태로 되어있는 것을 행 형태로 바꿀때 사용

## 데이터 분포의 변환

- 정규분포를 가정하는 분석 기법을 사용할 때 입력데이터가 정규를 따르지 않는 경우, 정규분포에 가깝게 변환하는 기법



# 사이킷런으로 수행하는 타이타닉 생존자 예측